In [1]:
from dotenv import load_dotenv

load_dotenv()

True

##### Helpers for pretty printing

In [15]:
import json

def obj_to_dict(obj):
    if isinstance(obj, list):
        return [obj_to_dict(e) for e in obj]
    elif hasattr(obj, "__dict__"):
        data = { 
            key: obj_to_dict(value) 
            for key, value in obj.__dict__.items() 
            if not key.startswith('_') and not callable(value)
        }
        return data
    else:
        return obj


def print_object(obj):
    print(json.dumps(obj_to_dict(obj), indent=4))

# Usage
# print(json.dumps(obj_to_dict(your_object), indent=4))


### Step 1: Setup the Tutor

In [2]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

### Step 2: Create a Thread

In [3]:
thread = client.beta.threads.create()

In [16]:
print_object(thread)

{
    "id": "thread_ajspmuYuc2UBVPzxDi2IvePU",
    "created_at": 1701229362,
    "metadata": {},
    "object": "thread"
}


### Step 3: Add a Message to Thread

In [10]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [17]:
print_object(message)

{
    "id": "msg_L4EZ4g5E65uT2odPhvRqnopu",
    "assistant_id": null,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "I need to solve the equation `3x + 11 = 14`. Can you help me?"
            },
            "type": "text"
        }
    ],
    "created_at": 1701229718,
    "file_ids": [],
    "metadata": {},
    "object": "thread.message",
    "role": "user",
    "run_id": null,
    "thread_id": "thread_ajspmuYuc2UBVPzxDi2IvePU"
}


### Step 4: Run the Assistant

In [18]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [19]:
print_object(run)

{
    "id": "run_zkOkjt0weQikgGezkyt8zSRH",
    "assistant_id": "asst_DnJv8t7aLjcdRancOVFQKyNL",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1701230867,
    "expires_at": 1701231467,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Jane Doe. The user has a premium account.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-1106-preview",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_ajspmuYuc2UBVPzxDi2IvePU",
    "tools": [
        {
            "type": "code_interpreter"
        }
    ]
}


### Step 5: Check the run status

In [20]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

### Step 6: Check the Assistant's response

In [21]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [23]:
print_object(messages)

{
    "data": [
        {
            "id": "msg_dUz3uSojnWINGYjz9sAjXmLS",
            "assistant_id": "asst_DnJv8t7aLjcdRancOVFQKyNL",
            "content": [
                {
                    "text": {
                        "annotations": [],
                        "value": "The solution to the equation `3x + 11 = 14` is `x = 1`, which matches our manual calculation. If you have any more equations or problems you need assistance with, feel free to ask!"
                    },
                    "type": "text"
                }
            ],
            "created_at": 1701230891,
            "file_ids": [],
            "metadata": {},
            "object": "thread.message",
            "role": "assistant",
            "run_id": "run_zkOkjt0weQikgGezkyt8zSRH",
            "thread_id": "thread_ajspmuYuc2UBVPzxDi2IvePU"
        },
        {
            "id": "msg_nljkhZr5n895QQxWoZOxp7SB",
            "assistant_id": "asst_DnJv8t7aLjcdRancOVFQKyNL",
            "content": [
   

In [24]:
for message in messages:
    role = message.role
    value = message.content
    print(f"{role} said {value}")

assistant said [MessageContentText(text=Text(annotations=[], value='The solution to the equation `3x + 11 = 14` is `x = 1`, which matches our manual calculation. If you have any more equations or problems you need assistance with, feel free to ask!'), type='text')]
assistant said [MessageContentText(text=Text(annotations=[], value="Certainly! To solve the equation `3x + 11 = 14`, you would perform the following steps:\n\n1. Subtract 11 from both sides of the equation to isolate the term with the variable (x) on one side:\n    `3x + 11 - 11 = 14 - 11`\n    which simplifies to:\n    `3x = 3`\n    \n2. Divide both sides of the equation by 3 to solve for x:\n    `3x / 3 = 3 / 3`\n    which simplifies to:\n    `x = 1`\n\nSo the solution to the equation `3x + 11 = 14` is `x = 1`. If you'd like, I can demonstrate that with a bit of Python code."), type='text')]
user said [MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type=